# Chapter 10 & 11

### Dan Wouden, Shijie Sheng, Mohan Sun, Alex van Oene


In [2]:
import option_pricer.option_class as oc
import option_pricer.binomial_option as bo
import numpy as np

from pprint import pprint

import warnings
warnings.filterwarnings('ignore')

import plotly.plotly as py
import plotly.graph_objs as go
import plotly

plotly.tools.set_credentials_file(username='avanoene', api_key='1Re82WRRlmXxV7iLiSX1')
plotly.offline.init_notebook_mode()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


## 10.1

In [3]:
def beta_delta(spot, option, pre_up, pre_down):
    delta = np.exp(-option.delta * option.h)*((pre_up - pre_down)/(spot*(option.upper - option.lower)))
    beta = np.exp(-option.r * option.h)*((option.upper*pre_down - option.lower*pre_up)/(option.upper - option.lower))
    return delta, beta

some_market_data = oc.MarketData(S_0=100, r=.08)
some_option = oc.Option(market_data=some_market_data, strike=105, T = .5, periods=1)

call = beta_delta(some_option.S_0, some_option, oc.callpayoff(some_option.upper*some_option.S_0, some_option.strike), 
                  oc.callpayoff(some_option.lower*some_option.S_0, some_option.strike))
put = beta_delta(some_option.S_0, some_option, oc.putpayoff(some_option.upper*some_option.S_0, some_option.strike),
                 oc.putpayoff(some_option.lower*some_option.S_0, some_option.strike))

call_premium = call[0]*some_option.S_0 + call[1]
put_premium = put[0]*some_option.S_0 + put[1]

print("For an european call, the premium is: {:.4f}, the delta is: {:.4f}, and the beta is {:.4f}".format(call_premium, call[0], call[1]))
print("For an european put, the premium is: {:.4f}, the delta is: {:.4f}, and the beta is {:.4f}".format(put_premium, put[0], put[1]))


For an european call, the premium is: 10.1722, the delta is: 0.5322, and the beta is -43.0458
For an european put, the premium is: 11.0551, the delta is: -0.4678, and the beta is 57.8371


## 10.4

In [4]:
import pandas as pd

MSFT = pd.read_csv("msft.csv", index_col=0)

MSFT = MSFT.loc[:, ["Close"]]
MSFT.index = pd.to_datetime(MSFT.index)

mask2016 = (MSFT.index >= "2016") & (MSFT.index < "2017")
mask2015 = (MSFT.index >= "2015") & (MSFT.index < "2016")
mask2014 = (MSFT.index >= "2014") & (MSFT.index < "2015")
mask2013 = (MSFT.index >= "2013") & (MSFT.index < "2014")
mask2012 = (MSFT.index >= "2012") & (MSFT.index < "2013")

MSFT2017 = MSFT.loc[(MSFT.index >= "2017")]
MSFT2016 = MSFT.loc[mask2016]
MSFT2015 = MSFT.loc[mask2015]
MSFT2014 = MSFT.loc[mask2014]
MSFT2013 = MSFT.loc[mask2013]
MSFT2012 = MSFT.loc[mask2012]


returns = np.zeros(len(MSFT))

for i in range(len(MSFT)-1):
    returns[i] = np.log(MSFT.iloc[i]/MSFT.iloc[i+1])
    
annualized_vol = np.sqrt(252)*returns.std() 

returns2016 = np.zeros(len(MSFT2016))
for i in range(len(MSFT2016)-1):
    returns2016[i] = np.log(MSFT2016.iloc[i]/MSFT2016.iloc[i+1])
    
returns2015 = np.zeros(len(MSFT2015))
for i in range(len(MSFT2015)-1):
    returns2015[i] = np.log(MSFT2015.iloc[i]/MSFT2015.iloc[i+1])
    
returns2014 = np.zeros(len(MSFT2014))
for i in range(len(MSFT2014)-1):
    returns2014[i] = np.log(MSFT2014.iloc[i]/MSFT2014.iloc[i+1])

returns2013 = np.zeros(len(MSFT2013))
for i in range(len(MSFT2013)-1):
    returns2013[i] = np.log(MSFT2013.iloc[i]/MSFT2013.iloc[i+1])
    
returns2012 = np.zeros(len(MSFT2012))
for i in range(len(MSFT2012)-1):
    returns2012[i] = np.log(MSFT2012.iloc[i]/MSFT2012.iloc[i+1])
    
annualized_vol_2012 = returns2012.std()*np.sqrt(252)
annualized_vol_2013 = returns2013.std()*np.sqrt(252)
annualized_vol_2014 = returns2014.std()*np.sqrt(252)
annualized_vol_2015 = returns2015.std()*np.sqrt(252)
annualized_vol_2016 = returns2016.std()*np.sqrt(252)

trace0 = go.Scatter(y = returns2012, mode = "lines", name = "2012 Returns")
trace1 = go.Scatter(y = returns2013, mode = "lines", name = "2013 Returns")
trace2 = go.Scatter(y = returns2014, mode = "lines", name = "2014 Returns")
trace3 = go.Scatter(y = returns2015, mode = "lines", name = "2015 Returns")
trace4 = go.Scatter(y = returns2016, mode = "lines", name = "2016 Returns")

data = [trace0, trace1, trace2, trace3, trace4]


plotly.offline.iplot(data, filename = "MSFT Returns")


In [5]:
trace5 = go.Scatter(
    x = [2012, 2013, 2014, 2015, 2016],
    y = [annualized_vol_2012, annualized_vol_2013, annualized_vol_2014, annualized_vol_2015, annualized_vol_2016],
    mode = "lines",
    name = "Annualized Volatility"
)

plotly.offline.iplot([trace5], filename = "MSFT Annualized Volatility")

In [6]:
annual_vol_2012_first = returns2012[0:(len(returns2012)/2+1)].std()*np.sqrt(252.0)
annual_vol_2012_second= returns2012[len(returns2012)/2:len(returns2012)].std()*np.sqrt(252.0)
annual_vol_2013_first = returns2013[0:(len(returns2013)/2+1)].std()*np.sqrt(252.0)
annual_vol_2013_second = returns2013[len(returns2013)/2:len(returns2013)].std()*np.sqrt(252.0)
annual_vol_2014_first = returns2014[0:(len(returns2014)/2+1)].std()*np.sqrt(252.0)
annual_vol_2014_second = returns2014[len(returns2014)/2:len(returns2014)].std()*np.sqrt(252.0)
annual_vol_2015_first = returns2015[0:(len(returns2015)/2+1)].std()*np.sqrt(252.0)
annual_vol_2015_second = returns2015[len(returns2015)/2:len(returns2015)].std()*np.sqrt(252.0)
annual_vol_2016_first = returns2016[0:(len(returns2016)/2+1)].std()*np.sqrt(252.0)
annual_vol_2016_second = returns2016[len(returns2016)/2:len(returns2016)].std()*np.sqrt(252.0)

vol = [annual_vol_2012_first, annual_vol_2012_second, 
       annual_vol_2013_first, annual_vol_2013_second, 
       annual_vol_2014_first, annual_vol_2014_second,
       annual_vol_2015_first, annual_vol_2015_second,
       annual_vol_2016_first, annual_vol_2016_second]

trace6 = go.Scatter(
    x = [2012, 2012.5, 2013, 2013.5, 2014, 2014.5, 2015, 2015.5, 2016, 2016.5],
    y = vol,
    mode = "lines",
    name = "Annualized Volatility Estmated Half Years",
)

plotly.offline.iplot([trace5, trace6], filename = "MSFT Annualized Volatility 2")

## 10.6

In [7]:


def binomial_tree(option, payoff):
    spot_tree = np.zeros([option.nodes, option.nodes])
    delta_tree = np.zeros([option.nodes, option.nodes])
    beta_tree = np.zeros([option.nodes, option.nodes])
    premium_tree = np.zeros([option.nodes, option.nodes])
    price_tree = np.zeros([option.nodes, option.nodes])
    payoff_tree = np.zeros([option.nodes, option.nodes])
    flat_prices = np.zeros([option.nodes, option.nodes])
    nodes_start = option.nodes
    
    while (option.nodes >= 1):
        nodes = option.nodes
        spots = oc.calc_spot(option)
        if len(spots) < len(spot_tree):
            spots = np.append(spots, np.zeros(len(spot_tree)-len(spots)))
        payoff_tree[:, (nodes - 1)] = payoff(spots, option.strike)
        premium_tree[:, (nodes - 1)] = payoff(spots, option.strike)
        spot_tree[:,(nodes-1)] = spots
        option.set_nodes(nodes - 1)
        
    option.set_nodes(nodes_start)
    disc = np.exp(-(option.r)*(option.T/option.periods))
    pu = option.prob * disc
    pd = (1-option.prob) * disc
    
    price_tree[:,option.periods] = payoff_tree[:,option.periods]
    flat_prices[:,option.periods] = payoff_tree[:,option.periods]
    
    for i in range(option.periods -1, -1, -1):
        for j in range(option.periods - 1, -1, -1):
            if spot_tree[j,i] == 0:
                beta_tree[j,i] = 0
                delta_tree[j,i] = 0
            else:
                beta_tree[j, i] = beta_delta(spot_tree[j,i], option, premium_tree[j,i+1], premium_tree[j+1,i+1])[1]
                delta_tree[j,i] = beta_delta(spot_tree[j,i], option, premium_tree[j,i+1], premium_tree[j+1,i+1])[0]
                premium_tree[j,i] = delta_tree[j,i]*spot_tree[j,i] + beta_tree[j,i]
                
                price_tree[j, i] = price_tree[j, i+1]*pu + price_tree[j+1, i+1]*pd
                flat_prices[j,i] = flat_prices[j, i+1]*pu + flat_prices[j+1, i+1]*pd
                if price_tree[j, i] < payoff(spot_tree[j, i], option.strike):
                    price_tree[j, i] = payoff(spot_tree[j, i], option.strike)
            
    return premium_tree, delta_tree, beta_tree, spot_tree, price_tree, payoff_tree, flat_prices

market_data = oc.MarketData(S_0=100, r=.08, sigma=.3, delta=0)
option_10_6 = oc.Option(market_data=market_data, strike= 95, T = 1, periods=2)

trees = binomial_tree(option_10_6, oc.callpayoff)
euro_price = bo.binomial_price(option = option_10_6, payoff = oc.callpayoff, euro = True)

np.set_printoptions(precision=4)
print("The premium tree is: \n", trees[0])
print("The delta tree is: \n",trees[1])
print("The beta tree is: \n",trees[2])
print("The European Price is", euro_price)

The premium tree is: 
 [[ 19.1105  37.4016  70.5767]
 [  0.       5.7264  13.3287]
 [  0.       0.       0.    ]]
The delta tree is: 
 [[ 0.712   1.      0.    ]
 [ 0.      0.3559  0.    ]
 [ 0.      0.      0.    ]]
The beta tree is: 
 [[-52.0859 -91.275    0.    ]
 [  0.     -24.2326   0.    ]
 [  0.       0.       0.    ]]
The European Price is 19.110539136


## 10.11

In [8]:
u = 1.2
d = 1.05
S0 = 100
K = 50
r = 7.696/100
T = 1
n = 1
pu = (np.exp(r * T/n)-d)/(u - d)


cu = oc.callpayoff(u * S0, K)
cd = oc.callpayoff(d * S0, K)

call_price = (cu*pu + cd*(1-pu))*np.exp(-r*T)
print(call_price)

53.7036555029


In [9]:
u = 1.4
d = .6

cu = oc.callpayoff(u * S0, K)
cd = oc.callpayoff(d * S0, K)

call_price = (cu*pu + cd*(1-pu))*np.exp(-r*T)
print(call_price)

24.0735438656


In [10]:
u = 1.4
d = .4

cu = oc.callpayoff(u * S0, K)
cd = oc.callpayoff(d * S0, K)

call_price = (cu*pu + cd*(1-pu))*np.exp(-r*T)
print(call_price)

16.6660593369


## 10.12

In [11]:
market_data = oc.MarketData(S_0=100, r=.08, sigma=.3, delta=0)
option_10_12 = oc.Option(market_data=market_data, strike= 95, T = 1, periods=3)

american_call_price = bo.binomial_price(payoff=oc.callpayoff, option=option_10_12, euro = False)
spot_tree = binomial_tree(option=option_10_12, payoff=oc.callpayoff)

print("The American call option price is {:.4f}".format(american_call_price))
print("The early excercise value is \n", oc.callpayoff(spot_tree[3], strike = 95))
print("The option value at each node is \n", spot_tree[0])

The American call option price is 18.2826
The early excercise value is 
 [[  5.      27.1246  54.1442  87.1418]
 [  0.       0.      10.4781  33.8147]
 [  0.       0.       0.       0.    ]
 [  0.       0.       0.       0.    ]]
The option value at each node is 
 [[ 18.2826  33.1493  56.6441  87.1418]
 [  0.       6.6897  15.0403  33.8147]
 [  0.       0.       0.       0.    ]
 [  0.       0.       0.       0.    ]]


In [12]:
euro_put_price = bo.binomial_price(payoff=oc.putpayoff, option=option_10_12, euro = True)

euro_price_parity = american_call_price - option_10_12.S_0*np.exp(-option_10_12.delta * option_10_12.T)+ np.exp(-option_10_12.r*option_10_12.T)*option_10_12.strike

print(euro_put_price, euro_price_parity)


5.9786051141 5.9786051141


In [13]:
american_put_price = bo.binomial_price(payoff=oc.putpayoff, option=option_10_12, euro = False)
print(american_put_price)

6.67790122713


## 10.13

In [14]:
market_data = oc.MarketData(S_0=100, r=0.08, sigma=0.3, delta=0.08)
option_10_13 = oc.Option(market_data=market_data, strike= 95, T = 1, periods=3)

american_call_price = bo.binomial_price(payoff=oc.callpayoff, option=option_10_13, euro = False)
spot_tree = binomial_tree(option=option_10_13, payoff=oc.callpayoff)

print("The American call option price is {:.4f}".format(american_call_price))
print("The early excercise value is \n", spot_tree[5])
print("The option value at each node is \n", spot_tree[4])

The American call option price is 20.7028
The early excercise value is 
 [[  5.      23.911   46.3982  73.1381]
 [  0.       0.       5.      23.911 ]
 [  0.       0.       0.       0.    ]
 [  0.       0.       0.       0.    ]]
The option value at each node is 
 [[ 19.1111  31.0855  48.8981  73.1381]
 [  0.       6.4747  12.4426  23.911 ]
 [  0.       0.       0.       0.    ]
 [  0.       0.       0.       0.    ]]


In [15]:
american_put_price = bo.binomial_price(payoff=oc.putpayoff, option=option_10_13, euro = False)
print(american_put_price)

spot_tree_put = binomial_tree(option=option_10_13, payoff=oc.putpayoff)

print(spot_tree_put[0])
print("Payoff \n", spot_tree_put[5])
print("Spot \n", spot_tree_put[3])
print("Price \n", spot_tree_put[4])

7.78836780515
[[  9.3259   3.0501   0.       0.    ]
 [ 95.      15.0676   5.7669   0.    ]
 [ 95.      95.      23.6389  10.9035]
 [ 95.      95.      95.      35.5251]]
Payoff 
 [[  0.       0.       0.       0.    ]
 [ 95.      10.9035   0.       0.    ]
 [ 95.      95.      24.2778  10.9035]
 [ 95.      95.      95.      35.5251]]
Spot 
 [[ 100.      118.911   141.3982  168.1381]
 [   0.       84.0965  100.      118.911 ]
 [   0.        0.       70.7222   84.0965]
 [   0.        0.        0.       59.4749]]
Price 
 [[  7.3209   2.2406   0.       0.    ]
 [  0.      13.5775   4.9427   0.    ]
 [  0.       0.      24.2778  10.9035]
 [  0.       0.       0.      35.5251]]


In [16]:
euro_call = bo.binomial_price(payoff=oc.callpayoff, option = option_10_13, euro = True)
euro_put = bo.binomial_price(payoff=oc.putpayoff, option = option_10_13, euro = True)

euro_price_parity = euro_call - option_10_13.S_0*np.exp(-option_10_13.delta * option_10_13.T)+ np.exp(-option_10_13.r*option_10_13.T)*option_10_13.strike


print(euro_call, euro_put, euro_price_parity)

20.7027969267 7.37409015924 16.0872151948


## 10.21

In [18]:
m_data_10_21 = oc.MarketData(S_0 = 100, sigma=.3, r=.05, delta=.03)
option_10_21 = oc.Option(m_data_10_21, strike=95, T=3, periods=3)

euro_call = bo.binomial_price(payoff=oc.callpayoff, option=option_10_21, euro = True)
euro_put = bo.binomial_price(payoff=oc.putpayoff, option=option_10_21, euro = True)

print(euro_call, euro_put)

m_data_10_21 = oc.MarketData(S_0 = 95, sigma=.3, r=.03, delta=.05)
option_10_21 = oc.Option(m_data_10_21, strike=100, T=3, periods=3)


euro_call = bo.binomial_price(payoff=oc.callpayoff, option=option_10_21, euro = True)
euro_put = bo.binomial_price(payoff=oc.putpayoff, option=option_10_21, euro = True)

print(euro_call, euro_put)

33.0274360335 13.0776383535
25.3121663737 21.1215679691


## 10.22

In [19]:
m_data_10_21 = oc.MarketData(S_0 = 100, sigma=.3, r=.05, delta=.03)
option_10_21 = oc.Option(m_data_10_21, strike=95, T=3, periods=3)

euro_call = bo.binomial_price(payoff=oc.callpayoff, option=option_10_21, euro = False)
euro_put = bo.binomial_price(payoff=oc.putpayoff, option=option_10_21, euro = False)

print(euro_call, euro_put)

m_data_10_21 = oc.MarketData(S_0 = 95, sigma=.3, r=.03, delta=.05)
option_10_21 = oc.Option(m_data_10_21, strike=100, T=3, periods=3)


euro_call = bo.binomial_price(payoff=oc.callpayoff, option=option_10_21, euro = False)
euro_put = bo.binomial_price(payoff=oc.putpayoff, option=option_10_21, euro = False)

print(euro_call, euro_put)

33.0274360335 14.0344175657
25.3121663737 21.2587017918


## 11.1

In [20]:
m_data_11_1 = oc.MarketData(S_0 = 100, sigma=.3, r=0.0, delta=.08)
k70 = oc.Option(m_data_11_1, strike=70, T=3, periods=3)
k80 = oc.Option(m_data_11_1, strike=80, T=3, periods=3)
k90 = oc.Option(m_data_11_1, strike=90, T=3, periods=3)

bintree70 = binomial_tree(k70, oc.callpayoff)
american70 = bo.binomial_price(oc.callpayoff, k70, euro = False)
print(american70)


47.0070474436


In [ ]:
## 11.1
s=100
r=0
sigma11=0.3
delta11=0.08
h=1
k1=70
k2=80
k3=90
k4=100

u=exp((r-delta11)*h+sigma11*sqrt(h))
d=exp((r-delta11)*h-sigma11*sqrt(h))
u
d

prob=(exp(r-delta11)*h-d)/(u-d)
prob
su=s*u
sd=s*d
su
sd
## k1 Early exercise happens when strike price is 70

cu=max(su-k1,0)
cd=max(sd-k1,0)
c=exp(-r*h)*(prob*cu+(1-prob)*cd)
cu
cd
c

pu=max(k1-su,0)
pd=max(k1-sd,0)
p=exp(-r*h)*(prob*pu+(1-prob)*pd)
p

## k2 Early exercise happens when strike price is 80

cu=max(su-k2,0)
cd=max(sd-k2,0)
c=exp(-r*h)*(prob*cu+(1-prob)*cd)
cu
cd
c

pu=max(k2-su,0)
pd=max(k2-sd,0)
p=exp(-r*h)*(prob*pu+(1-prob)*pd)
p

## k3 Early exercise will not happen at strike price is 90

cu=max(su-k3,0)
cd=max(sd-k3,0)
c=exp(-r*h)*(prob*cu+(1-prob)*cd)
cu
cd
c

pu=max(k3-su,0)
pd=max(k3-sd,0)
p=exp(-r*h)*(prob*pu+(1-prob)*pd)
p

## k4 Early exercise will not happen when strike price is 100.

cu=max(su-k4,0)
cd=max(sd-k4,0)
c=exp(-r*h)*(prob*cu+(1-prob)*cd)
cu
cd
c
pu=max(k4-su,0)
pd=max(k4-sd,0)
p=exp(-r*h)*(prob*pu+(1-prob)*pd)
p

##11.1.b.Put call Parity
## $Call=S_{0}e^{-\delta*T}-Ke^{-r}+Put$ After calculate $S_{0}e^{-\delta}-Ke^{-r}$

pcparity=s*exp(-delta11)
pcparity


## Then the equation will be $Call=92.31163-K+Put$. From the equation we can get the exercise happens when put price is less than 7.68837 or 100-k, because $100-k$ is larger than $92.31163-k$. From the calculation above, the put price is less than 7.68837 at strkie price is 90. So, people will exercise the call option early.

##11.1.c
## From the result of put price we can get, higher strike price will lead higher put price. The strike price that has lower put price will exercise the option earlier.

##11.2

s=100
r=0.08
sigma11=0.3
delta11=0.08
h=1
k1=70
k2=80
k3=90
k4=100

u=exp((r-delta11)*h+sigma11*sqrt(h))
d=exp((r-delta11)*h-sigma11*sqrt(h))
prob=(exp(r-delta11)*h-d)/(u-d)
u
d
prob

su=s*u
sd=s*d
su
sd

## k1 Early exercise will happen when strike price is 70

cu=max(su-k1,0)
cd=max(sd-k1,0)
c=exp(-r*h)*(prob*cu+(1-prob)*cd)
cu
cd
c

pu=max(k1-su,0)
pd=max(k1-sd,0)
p=exp(-r*h)*(prob*pu+(1-prob)*pd)
p

## k2 Early exercise will not happen when strike price is 80.

cu=max(su-k2,0)
cd=max(sd-k2,0)
c=exp(-r*h)*(prob*cu+(1-prob)*cd)
cu
cd
c

pu=max(k2-su,0)
pd=max(k2-sd,0)
p=exp(-r*h)*(prob*pu+(1-prob)*pd)
p

## k3 Early exercise will not happen when strike price is 90.

cu=max(su-k3,0)
cd=max(sd-k3,0)
c=exp(-r*h)*(prob*cu+(1-prob)*cd)
cu
cd
c

pu=max(k3-su,0)
pd=max(k3-sd,0)
p=exp(-r*h)*(prob*pu+(1-prob)*pd)
p

## k4 Early exercise will not happen when strike price is 100.

cu=max(su-k4,0)
cd=max(sd-k4,0)
c=exp(-r*h)*(prob*cu+(1-prob)*cd)
cu
cd
c

pu=max(k4-su,0)
pd=max(k4-sd,0)
p=exp(-r*h)*(prob*pu+(1-prob)*pd)
p
 
## $Call=S_{0}e^{-\delta*T}-Ke^{-r}+Put$ After calculate $S_{0}e^{-\delta}-Ke^{-r}$

t=1
pcparity=s*exp(-delta11*t)
pcparity
pcparity1=exp(-r*t)
pcparity1
```
## From the result, we also observe the put price goes higher with the strike price increases. The $Ke^{-rT}$ will be just k and the $S_{0}e^{-\delta*T}$ will be just spot price when immediate exercise happens(T=0). Then we get $S_{0}-K$ which is $Call=100-K+Put$. When T=1, the equation will be $92.31163-K*0.9231163+Put$. The early exercise will be excecute when $100-k>92.31163-K*0.9231163+Put$, then we get $Put<7.68837-0.0768837k$ 

##11.3

s=100
r=0.08
sigma11=0.3
delta11=0
h=1
k1=70
k2=80
k3=90
k4=100

u=exp((r-delta11)*h+sigma11*sqrt(h))
d=exp((r-delta11)*h-sigma11*sqrt(h))
prob=(exp(r-delta11)*h-d)/(u-d)
u
d
prob

su=s*u
sd=s*d
su
sd

## k1 Early exercise will not happen when strike price is 70

cu=max(su-k1,0)
cd=max(sd-k1,0)
c=exp(-r*h)*(prob*cu+(1-prob)*cd)
cu
cd
c

pu=max(k1-su,0)
pd=max(k1-sd,0)
p=exp(-r*h)*(prob*pu+(1-prob)*pd)
p

## k2 Early exercise will not happen when strike price is 80.

cu=max(su-k2,0)
cd=max(sd-k2,0)
c=exp(-r*h)*(prob*cu+(1-prob)*cd)
cu
cd
c

pu=max(k2-su,0)
pd=max(k2-sd,0)
p=exp(-r*h)*(prob*pu+(1-prob)*pd)
p

## k3 Early exercise will not happen when strike price is 90.

cu=max(su-k3,0)
cd=max(sd-k3,0)
c=exp(-r*h)*(prob*cu+(1-prob)*cd)
cu
cd
c

pu=max(k3-su,0)
pd=max(k3-sd,0)
p=exp(-r*h)*(prob*pu+(1-prob)*pd)
p

## k4 Early exercise will not happen when strike price is 100.

cu=max(su-k4,0)
cd=max(sd-k4,0)
c=exp(-r*h)*(prob*cu+(1-prob)*cd)
cu
cd
c

pu=max(k4-su,0)
pd=max(k4-sd,0)
p=exp(-r*h)*(prob*pu+(1-prob)*pd)
p

## There will be no early exercise happen when \delta=0. Investors will get benefit from dividend when exercise the option earlier to get the stock. From the equation, since \delta=0, $S_{0}e^{-\delta*T}=S_{0}$.

##11.4

s=100
r=0.08
sigma11=0.3
delta11=0
h=1
k1=100
k2=110
k3=120
k4=130


u=exp((r-delta11)*h+sigma11*sqrt(h))
d=exp((r-delta11)*h-sigma11*sqrt(h))
prob=(exp(r-delta11)*h-d)/(u-d)
u
d
prob

su=s*u
sd=s*d
su
sd

## k1 Early exercise will not happen when strike price is 100

cu=max(su-k1,0)
cd=max(sd-k1,0)
c=exp(-r*h)*(prob*cu+(1-prob)*cd)
cu
cd
c

pu=max(k1-su,0)
pd=max(k1-sd,0)
p=exp(-r*h)*(prob*pu+(1-prob)*pd)
p
ppayoff=max(k1-s-p,0)
ppayoff

## k2 Early exercise will not happen when strike price is 110.

cu=max(su-k2,0)
cd=max(sd-k2,0)
c=exp(-r*h)*(prob*cu+(1-prob)*cd)
cu
cd
c

pu=max(k2-su,0)
pd=max(k2-sd,0)
p=exp(-r*h)*(prob*pu+(1-prob)*pd)
p
ppayoff=max(k2-s-p,0)
ppayoff

## k3 Early exercise will not happen when strike price is 120.

cu=max(su-k3,0)
cd=max(sd-k3,0)
c=exp(-r*h)*(prob*cu+(1-prob)*cd)
cu
cd
c

pu=max(k3-su,0)
pd=max(k3-sd,0)
p=exp(-r*h)*(prob*pu+(1-prob)*pd)
p
ppayoff=max(k3-s-p,0)
ppayoff

## k4 Early exercise will happen when strike price is 130.

cu=max(su-k4,0)
cd=max(sd-k4,0)
c=exp(-r*h)*(prob*cu+(1-prob)*cd)
cu
cd
c

pu=max(k4-su,0)
pd=max(k4-sd,0)
p=exp(-r*h)*(prob*pu+(1-prob)*pd)
p
ppayoff=max(k4-s-p,0)
ppayoff

pcparity=exp(-r*1)
pcparity
pcparity1=s*exp(-delta11*1)
pcparity1


##11.4.b. 
# The function for put-call parity is $Put=Ke^{-rT}-Se^{-\delta*T}+Call$. In order to exercise early, the put-call parity should match $Put>Ke^{-rT}-Se^{-\delta*T}+Call$. Then we get $K-100>0.9231163K-100+Call$, $call<0.07688k$. The situation that will fulfill either of these two function will be exercised early.
##11.4.c
# From the result above we can see the call price decrease with the strike price increases. Combine the answer from question b, the call price decreases means the early exercise will happen lower than 100.

##11.7.a

s=100
k=100
sigma=0.3
r=0.08
t=1
deltaa=0
n=10
h=t/n



## 11.8

In [21]:
m_data_11_8 = market_data=oc.MarketData(S_0=100.0, sigma= .3, r = .08, delta = 0.0)
k70 = oc.Option(m_data_11_8, strike = 70, T = 1, periods=10)
k80 = oc.Option(m_data_11_8, strike = 80, T = 1, periods=10)
k90 = oc.Option(m_data_11_8, strike = 90, T = 1, periods=10)
k100 = oc.Option(m_data_11_8, strike = 100, T = 1, periods=10)
k110 = oc.Option(m_data_11_8, strike = 110, T = 1, periods=10)
k120 = oc.Option(m_data_11_8, strike = 120, T = 1, periods=10)
k130 = oc.Option(m_data_11_8, strike = 130, T = 1, periods=10)


price70 = bo.binomial_price(oc.callpayoff, k70, euro = True)
delta70 = binomial_tree(k70, oc.callpayoff)[1][0,0]
beta70 = binomial_tree(k70, oc.callpayoff)[2][0,0]

price80 = bo.binomial_price(oc.callpayoff, k80, euro = True)
delta80 = binomial_tree(k80, oc.callpayoff)[1][0,0]
beta80 = binomial_tree(k80, oc.callpayoff)[2][0,0]

price90 = bo.binomial_price(oc.callpayoff, k90, euro = True)
delta90 = binomial_tree(k90, oc.callpayoff)[1][0,0]
beta90 = binomial_tree(k90, oc.callpayoff)[2][0,0]

price100 = bo.binomial_price(oc.callpayoff, k100, euro = True)
delta100 = binomial_tree(k100, oc.callpayoff)[1][0,0]
beta100 = binomial_tree(k100, oc.callpayoff)[2][0,0]

price110 = bo.binomial_price(oc.callpayoff, k110, euro = True)
delta110 = binomial_tree(k110, oc.callpayoff)[1][0,0]
beta110 = binomial_tree(k110, oc.callpayoff)[2][0,0]

price120 = bo.binomial_price(oc.callpayoff, k120, euro = True)
delta120 = binomial_tree(k120, oc.callpayoff)[1][0,0]
beta120 = binomial_tree(k120, oc.callpayoff)[2][0,0]

price130 = bo.binomial_price(oc.callpayoff, k130, euro = True)
delta130 = binomial_tree(k130, oc.callpayoff)[1][0,0]
beta130 = binomial_tree(k130, oc.callpayoff)[2][0,0]

print("For a strike of 70, the price is: \n", price70)
print("For a strike of 70, the delta is: \n", delta70)
print("For a strike of 70, the beta is: \n", beta70)

print("For a strike of 80, the price is: \n", price80)
print("For a strike of 80, the delta is: \n", delta80)
print("For a strike of 80, the beta is: \n", beta80)

print("For a strike of 90, the price is: \n", price90)
print("For a strike of 90, the delta is: \n", delta90)
print("For a strike of 90, the beta is: \n", beta90)

print("For a strike of 100, the price is: \n", price100)
print("For a strike of 100, the delta is: \n", delta100)
print("For a strike of 100, the beta is: \n", beta100)

print("For a strike of 110, the price is: \n", price110)
print("For a strike of 110, the delta is: \n", delta110)
print("For a strike of 110, the beta is: \n", beta110)

print("For a strike of 120, the price is: \n", price120)
print("For a strike of 120, the delta is: \n", delta120)
print("For a strike of 120, the beta is: \n", beta120)

print("For a strike of 130, the price is: \n", price130)
print("For a strike of 130, the delta is: \n", delta130)
print("For a strike of 130, the beta is: \n", beta130)

For a strike of 70, the price is: 
 36.1365692549
For a strike of 70, the delta is: 
 0.94900948615
For a strike of 70, the beta is: 
 -58.7643793601
For a strike of 80, the price is: 
 28.3434797558
For a strike of 80, the delta is: 
 0.877866849394
For a strike of 80, the beta is: 
 -59.4432051837
For a strike of 90, the price is: 
 21.1641724762
For a strike of 90, the delta is: 
 0.785600905237
For a strike of 90, the beta is: 
 -57.3959180475
For a strike of 100, the price is: 
 15.9622023316
For a strike of 100, the delta is: 
 0.661504460006
For a strike of 100, the beta is: 
 -50.188243669
For a strike of 110, the price is: 
 11.1356593559
For a strike of 110, the delta is: 
 0.539285150619
For a strike of 110, the beta is: 
 -42.7928557061
For a strike of 120, the price is: 
 8.18001655552
For a strike of 120, the delta is: 
 0.42642034211
For a strike of 120, the beta is: 
 -34.4620176554
For a strike of 130, the price is: 
 5.22437375518
For a strike of 130, the delta is: 
 

In [29]:
def real_probs(option, alpha, payoff):
    h = option.T/option.periods
    u = np.exp((option.r - option.delta)*h + option.sigma*np.sqrt(h))
    d = np.exp((option.r - option.delta)*h - option.sigma*np.sqrt(h))
    pu = (np.exp(alpha * h) - u)/(u - d)
    pd = 1 - pu
    disc = np.exp(-option.r * h)
    dpu = disc * pu
    dpd = disc * pd
    
    Ct = np.zeros(option.nodes)
    St = np.zeros(option.nodes)
    gamma = np.zeros(option.nodes)
    premium_tree = np.zeros(option.nodes)
    
    for i in range(option.nodes):
        St[i] = option.S_0 * (u ** (option.periods - i)) * (d ** i)
        Ct[i] = payoff(St[i], option.strike)
    
    for i in range((option.periods - 1), -1, -1):
        for j in range(i+1):
            delta = beta_delta(St[j], option, Ct[j], Ct[j+1])[0]
            beta = beta_delta(St[j], option, Ct[j], Ct[j+1])[1]
            premium = delta * St[j] + beta
            premium_tree[j] = premium
            gamma[j] = np.log((delta*St[j])/(premium)*np.exp(alpha * h) + (beta)/(premium)*np.exp(option.r * h))
            Ct[j] = dpu * Ct[j] + dpd * Ct[j+1]
            St[j] = St[j] / u
            Ct[j] = np.maximum(Ct[j], payoff(St[j], option.strike))
            
    return gamma*option.periods, Ct, St, premium_tree

real_probs(k80, .15, oc.callpayoff)[0]

array([ 0.4147,  0.5896,  0.8275,  0.8275,     nan,     nan,     nan,
           nan,     nan,     nan,  0.    ])